# IML 2018 Modeled

In [1]:
dataset_id = "iml_2018_modeled"

In [2]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'sashimi-266523:iml_2018_modeled' successfully created.


## Tables from Original Public Resources Table

In [11]:
%%bigquery
create table iml_2018_modeled.public_resources as
select MID, DISCIPL, COMMONNAME, LEGALNAME, ADSTREET, ADCITY, ADSTATE, 
cast(ADZIP5 as STRING) as ZIPCODE, 
PHONE, WEBURL, 
cast(replace(INCOME15, ' ', '0') as FLOAT64) as INCOME, 
cast(replace(REVENUE15, ' ', '0') as FLOAT64) as REVENUE, 
LONGITUDE, LATITUDE
from `iml_2018_staging.public_resources` 

""


In [66]:
%%bigquery
select count(*) from `iml_2018_modeled.public_resources`  

,f0_
0,7429


In [67]:
%%bigquery
select count(distinct MID) from `iml_2018_modeled.public_resources`  

,f0_
0,7429


# Process Data with Beam

In [2]:
% run Public_Resources_Beam.py

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'iml_2018_modeled'
 projectId: 'sashimi-266523'
 tableId: 'public_resources'> referenced by query SELECT * FROM iml_2018_modeled.public_resources limit 50
INFO:apache_beam.io.gcp.bigquery_tools:Created table sashimi-266523.iml_2018_modeled.Public_Resources_Beam with schema <TableSchema
 fields: [<TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'MID'
 type: 'INTEGER'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'DISCIPL'
 type: 'STRING'>, <TableFieldSchema
 fields: []
 mode: 'NULLABLE'
 name: 'COMMONNAME'
 type: 'STR